In [351]:
import numpy as np
import pandas as pd
from datetime import datetime

import matplotlib
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
matplotlib.rcParams['axes.labelsize'] = 14
matplotlib.rcParams['xtick.labelsize'] = 12
matplotlib.rcParams['ytick.labelsize'] = 12
matplotlib.rcParams['text.color'] = 'k'

import warnings
warnings.filterwarnings("ignore")
from sklearn.utils.testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning


from scipy.stats.stats import pearsonr



## Configurable parameters

# Make True to see plots while finding related features
SHOW_PLOTS = False

# Make True to see feature importance graphs
SHOW_FEATURE_IMPORTANCE = False

SyntaxError: invalid syntax (<ipython-input-351-4dcd207083af>, line 21)

In [236]:
def read_file():
    '''
    Reads the challenge data 'Sandesh' and preprocess it:
        -tranpose the raw data
        -set datetime as index 
        -set 'Generic_LookupKeys' as columns
        -remove descriptive rows
        
    Input: NA.
    Output: full_data, target_data(with only target variables)
    
    '''
    
    data_df = pd.read_excel('data/ChallengeData-Sandesh.xlsx')
    
    #save the dates for indexing
    date_index = data_df.iloc[1,9:].dt.strftime('%Y/%m')
    
    #find the interested variables
    target_rows = [25, 26, 76, 77, 79, 80, 82, 83, 85, 86, 91, 92]
    
    target_data = data_df.iloc[[x-2 for x in target_rows], :]
    Target_Generic_LookupKeys = list(target_data.iloc[:,7])
    
    #dropping the descriptive rows and reset index and columns
    target_data = target_data.T.iloc[9:,:]
    target_data.set_index(date_index, inplace=True)
    target_data.columns = Target_Generic_LookupKeys
    target_data.dropna(how='all', inplace=True)
    
    #dataframe with all the feature lookupkeys
    Generic_LookupKeys = list(data_df.iloc[2:,7])
    full_data = data_df.iloc[2:,:].T.iloc[9:,:]
    full_data.set_index(date_index, inplace=True)
    full_data.columns = Generic_LookupKeys
    full_data.dropna(how='all', inplace=True)
    
    return full_data, target_data

# Related features

In [246]:
def find_correlated_features(full_data, target_data):
    '''
    Checks correlation between every pair of features and keeps a list of highest correlated features for every feature.
    These correlated features are used in modelling.
    Input: submission and train dataframes.
    Output: A dictionary of correlated features for every feature.
    '''
    
    related_features = {}
    related_features_names = []
    related_features_correlations = []
    
    Target_Generic_LookupKeys = target_data.columns.tolist()
    Generic_LookupKeys = full_data.columns.tolist()
    
    
    # For every lookupkey in target_data, find the correlated features
    for generic_lookupkey in Target_Generic_LookupKeys:
        #print(generic_lookupkey)
        
        # Initialize the related features list for this feature
        related_features[generic_lookupkey] = []
        generic_lookups = Generic_LookupKeys.copy()
        
        # Remove the currenyt feature itself from all the other features
        if generic_lookupkey in generic_lookups:
            generic_lookups.remove(generic_lookupkey)
        
        list_series = []
        list_series.append(full_data[generic_lookupkey].dropna().tolist())
    
        # For all the other features, find correlation coefficient with the current feature
        for generic_lookup in generic_lookups:
            list_series.append(full_data[generic_lookup].dropna().tolist())
            if SHOW_PLOTS and len(list_series[0]) > 0 and len(list_series[-1]) > 0:
                initial_analysis_plots([list_series[0], list_series[-1]], [generic_lookupkey, generic_lookup])
            min_len = min(len(list_series[0]), len(list_series[-1]))
            if min_len > 1:
                p_coeff = pearsonr(list_series[0][-min_len:], list_series[-1][-min_len:])[0]
                # print(p_coeff)
                # If correlation coefficient is more than 0.8 or less than -0.8, include this feature
                if p_coeff > 0.8 or p_coeff < -0.8:
                    related_features[generic_lookupkey].append(generic_lookup)
                    related_features[generic_lookupkey].append(generic_lookup)
                    related_features_names.append(generic_lookup)
                    related_features_correlations.append(p_coeff)

                
        related_features_df = pd.DataFrame()
        related_features_df['features'] = related_features_names
        related_features_df['correlation'] = np.abs(related_features_correlations)
        
        if SHOW_FEATURE_IMPORTANCE and related_features_df.shape[0] > 0:
            ax = related_features_df.plot(x='features', y='correlation', kind='bar', title=generic_lookupkey, figsize=(12, 8), legend=True, fontsize=12)
            ax.set_xlabel('feature name', fontsize=12)
            ax.set_ylabel('correlation value', fontsize=12)
            plt.show()
                    
    return related_features

In [347]:
def join_related_features(df, train, related_features, generic_lookup):
    '''
    Joins all the correlated features to the current feature (generic lookup). Since, length of data could be different 
    for all the features so overall length of dataframe is also tracked.
    Input:  df- A dataframe having only current generic_lookup key data as single column.
            train- train dataframe.
            related_features- dictionary of related features for every lookupkey.
            generic_lookup- the generic lookup in consideration.
    Output: A dataframe having all the correlated features to the current generic lookup.
    '''
    smallest_series_length = df.shape[0]
    max_other_series_name = 0
    max_other_series_length = -1
    
    #print('number of related series: ', len(related_features[generic_lookup]))
    for related_feature in related_features[generic_lookup]:
        #print(related_feature)
        series_data = full_data[related_feature].tolist()
        #print('length of series data: ', len(series_data))
        if len(series_data) >= smallest_series_length:
            df[related_feature] = series_data[-smallest_series_length:]
        elif len(series_data) > max_other_series_length:
            max_other_series_name = related_feature
            max_other_series_length = len(series_data)
            
    return df

# Helper Functions

In [339]:
'''
HELPER FUNCTIONS
'''

import itertools
import statsmodels.api as sm


#Quantitative Scoring using MAPE
def MAPE(gt, pred):
    mape = []

    for g, p in zip(gt, pred):
        mape.append(max(0, 1 - abs((g-p)/g)))

    return np.mean(mape)




def best_parameters_sarimax(series):
    '''
    Finds the best parameters for a given series for SARIMAX algorithm.
    Input: series: the series for which the parameters are to be determined.
    Output: the best parameters for the series and model.
    '''
    result_param = -1
    result_param_seasonal = -1
    p = d = q = range(0, 2)
    pdq = list(itertools.product(p, d, q))
    seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]
    for param in pdq:
        for param_seasonal in seasonal_pdq:
            try:
                mod = sm.tsa.statespace.SARIMAX(series,
                                                order=param,
                                                seasonal_order=param_seasonal,
                                                enforce_stationarity=False,
                                                enforce_invertibility=False)
                results = mod.fit()
                #print('ARIMA{}x{}12 - AIC:{}'.format(param, param_seasonal, results.aic))
                try:
                    if results.aic < minimum:
                        result_param = param
                        result_param_seasonal = param_seasonal
                except:
                    result_param = param
                    result_param_seasonal = param_seasonal
                    minimum = results.aic
            except:
                continue
    return result_param, result_param_seasonal

def apply_model_sarimax(series, best_param, best_param_seasonal):
    '''
    Makes and trains SARIMAX model on the given series and parameters.
    Input:  series: series on which the model is to be trained.
            best_param, best_param_seasonal: best parameters for modelling
    Output: Trained model
    '''
    mod = sm.tsa.statespace.SARIMAX(series,
                                    order=best_param,
                                    seasonal_order=best_param_seasonal,
                                    enforce_stationarity=False,
                                    enforce_invertibility=False)
    results = mod.fit()
    return results

# Models

In [342]:
def train_and_predict_varmax(train_df, generic_lookup, validation_values):
    '''
    Train a VARMAX and predicts on the validation data.
    Input:  train_df- training dataframe.
            generic_lookup- the generic_llokup for which we are going to predict.
            validation_values- the validation set to compare with.
    Output: The validation set RMSE value.
    '''
    try:
        model = VARMAX(train_df[:-5], order=(2, 0), trend='c', enforce_stationarity=False, enforce_invertibility=False)
        model_result = model.fit(maxiter=1000, disp=False)
        validation_forecast = model_result.forecast(steps=5)[generic_lookup].tolist()
        mape_varmax = MAPE(validation_values, validation_forecast)
        return mape_varmax
    except:
        return -1

In [343]:
def train_and_predict_sarimax(train_df, generic_lookup, validation_values):
    '''
    Train a SARIMAX and predicts on the validation data.
    Input:  train_df- training dataframe.
            generic_lookup- the generic_llokup for which we are going to predict.
            validation_values- the validation set to compare with.
    Output: The validation set RMSE value.
    '''
    try:
        best_param, best_param_seasonal = best_parameters_sarimax(train_df[generic_lookup][:-5])
        results = apply_model_sarimax(train_df[generic_lookup][:-5], best_param, best_param_seasonal)
        forecast = results.get_forecast(steps=5)
        validation_forecast = forecast.predicted_mean.tolist() # What is predicted_mean????
        
        #Computing validation metrics MAPE
        mape_sarimax = MAPE(validation_values, validation_forecast)
        return mape_sarimax
    except:
        return -1

In [352]:
@ignore_warnings(category=ConvergenceWarning)
def cross_validation_and_model_comparison(train_df, number_of_steps_to_predict, generic_lookup, validation_values):
    '''
    Compares the three models' validation MAPE values and returns the predictions for the best model.
    Input:  train_df- the dataframe of current generic lookup for training and prediction.
            train- full training dataframe.
            number_of_steps_to_predict- number of future steps to predict.
            validation_values- the validation set for comparison.
    Output: The final forecast for the current generic lookup.
    '''

    # Check validation MAPE for all three the models
    validation_mape_varmax = train_and_predict_varmax(train_df, generic_lookup, validation_values)
    validation_mape_sarimax = train_and_predict_sarimax(train_df, generic_lookup, validation_values)
    #validation_mape_LSTM = train_and_predict_LSTM()
    
    if validation_mape_varmax != 1 and validation_mape_sarimax != 1:
        print('validation_mape_varmax:', validation_mape_varmax)
        print('validation_mape_sarimax:', validation_mape_sarimax)
        #print('validation_mape_LSTM:', validation_mape_LSTM)

    # If validation RMSE of VARMAX is lesser than SARIMAX, use VARMAX else SARIMAX
    if validation_mape_varmax != 1 and validation_mape_varmax < validation_mape_sarimax:
        try:
            model = VARMAX(train_df, order=(2, 0), trend='c', enforce_stationarity=False, enforce_invertibility=False)
            model_result = model.fit(maxiter=1000, disp=False)
            forecast = model_result.forecast(steps=number_of_steps_to_predict)
            forecast = forecast[generic_lookup].tolist()
            return forecast
        except:
            best_param, best_param_seasonal = best_parameters_sarimax(train_df[generic_lookup])
            results = apply_model_sarimax(train_df[generic_lookup], best_param, best_param_seasonal)
        #   print(results.summary().tables[1])
        #   plt.show()
            forecast = results.get_forecast(steps=number_of_steps_to_predict)
            pred_ci = forecast.conf_int()
            forecast = forecast.predicted_mean.tolist()
            return forecast
    
    elif validation_mape_sarimax != 1:
        best_param, best_param_seasonal = best_parameters_sarimax(train_df[generic_lookup])
        results = apply_model_sarimax(train_df[generic_lookup], best_param, best_param_seasonal)
    #   print(results.summary().tables[1])
    #   plt.show()
        forecast = results.get_forecast(steps=number_of_steps_to_predict)
        pred_ci = forecast.conf_int()
        forecast = forecast.predicted_mean.tolist()
        return forecast
        
    return 1

NameError: name 'ignore_warnings' is not defined

In [345]:
# def get_number_of_steps_to_predict(submission, generic_lookup):
#     '''
#     Returns the number of future data points for which the predictions are to be made.
#     Input:  submission: the submission file
#             generic_lookup: the generic llokup for which the predictions are to be made
#     Output: the number of future time steps for which the predictions are to be made.
#     '''
#     return submission[submission['Generic LookupKey'] == generic_lookup].shape[0]
##################
# REPLACING



def make_predictions(target_data, full_data, related_features):
    '''
    The parent function to join correlated features, get number of steps, does cross validation and compares models.
    Input:  submission: submission dataframe.
            train: train dataframe.
            related_features: dictionary of related features.
    Output: Full forecast for all the generic lookup.
    '''
    # Initialize a list for all generic lookup.
    full_forecast = {}
    
    
    # For every generic lookup in submission.
    for generic_lookup in target_data.columns:
        
        print('generic lookup: ', generic_lookup)
        train_df = pd.DataFrame()
        series_data = target_data[generic_lookup].dropna().tolist()
        train_df[generic_lookup] = series_data

        # Join all correlated features to dataframe of this generic lookup
        train_df = join_related_features(train_df, full_data, related_features, generic_lookup)
        #mostly only half the number of feature are kept
        #print("number of related usable features: {}".format(train_df.shape[0]))
  

        # Get number of steps to predict for this generic lookup
        number_of_steps_to_predict = 12

        # Take last 5 as validation
        validation_values = train_df[generic_lookup][-5:]
    
    
        # Check cross validation and do final prediction.
        forecast = cross_validation_and_model_comparison(train_df, number_of_steps_to_predict, generic_lookup, validation_values)
        if forecast == 1:
            print("TERRIBLEEEEEEEEEEEEEEEE")
            #will mean work better??????
            forecast = [train_df[generic_lookup].mean()]*number_of_steps_to_predict
        full_forecast[generic_lookup] = forecast
    
    return full_forecast

# Main

In [353]:
if __name__ == '__main__':
    print('Reading files')
    full_data, target_data = read_file()
    print('Finding related features')
    related_features = find_correlated_features(full_data, target_data)
    print('Making full forecast')
    full = make_predictions(target_data, full_data, related_features)
#     submission['Value'] = full_forecasts
#     submission.to_csv('../output/submission.csv', index=False)

Reading files
Finding related features
Segment 1 - Sandesh Brand 2Sandesh Brand 2BroadbandTortoiseRevenue - Tortoise
Segment 1 - Sandesh Brand 2Sandesh Brand 2BroadbandRabbitRevenue - Rabbit
Segment 1 - Sandesh Brand 2Sandesh Brand 2BroadbandTortoiseAverage Revenue per existing customer Excl Line Rental - Tortoise
Segment 1 - Sandesh Brand 2Sandesh Brand 2BroadbandRabbitAverage Revenue per existing customer Excl Line Rental - Rabbit
Segment 1 - Sandesh Brand 2Sandesh Brand 2BroadbandTortoiseLeavers - Tortoise
Segment 1 - Sandesh Brand 2Sandesh Brand 2BroadbandRabbitLeavers - Rabbit 
Segment 1 - Sandesh Brand 2Sandesh Brand 2BroadbandTortoiseClosing Base - Tortoise
Segment 1 - Sandesh Brand 2Sandesh Brand 2BroadbandRabbitClosing Base - Rabbit 
Segment 1 - Sandesh Brand 2Sandesh Brand 2BroadbandTortoiseGross Adds - Tortoise
Segment 1 - Sandesh Brand 2Sandesh Brand 2BroadbandRabbitGross Adds - Rabbit 
Segment 1 - Sandesh Brand 2Sandesh Brand 2BroadbandTortoiseNet Migrations - Tortoise
Seg

# Trial

In [286]:
gt = [2.5, 3.2, 4.7]
pred = [3, 3.5, 5.2]

mape = []

for g, p in zip(gt, pred):
    mape.append(max(0, 1 - abs(g-p)/g))

return np.mean(mape)

In [287]:
mape

[0.8, 0.90625, 0.8936170212765957]

In [290]:
np.mean(mape)

0.8666223404255319

In [292]:
MAPE(gt, pred)

0.8666223404255319

# extend datetime data

In [313]:
time = datetime.strptime('2010-8', '%Y-%m').date()
time1 = datetime.strptime('2010-9', '%Y-%m').date()
diff = time1 - time

In [318]:
time2 = datetimetime1 + diff

AttributeError: 'datetime.date' object has no attribute 'date'

In [317]:
time2

datetime.date(2010, 10, 2)

In [314]:
type(diff)

datetime.timedelta

In [315]:
diff

datetime.timedelta(31)